In [1]:
# 山东大学 信息检索实验 
'''
以 山东大学新闻网为起点进行网页的循环爬取，保持爬虫在view.sdu.edu.cn之内(既只爬取这个站点的网页)，爬取的网页数量越多越好。
'''

'\n以 山东大学新闻网为起点进行网页的循环爬取，保持爬虫在view.sdu.edu.cn之内(既只爬取这个站点的网页)，爬取的网页数量越多越好。\n'

In [2]:
import re
import requests
from bs4 import BeautifulSoup
import bs4
import os
import codecs

In [3]:
test_version = False

savepath = r"G:/trash/news"

MAX_NUM_NEWS =  10000

In [4]:
# 遍历过的URL
visited_URL_set = set()

time_pattern = re.compile(r"(\d{4}年\d{2}月\d{2}日.*?\d{2}\:\d{2})") #发布日期：2018年05月07日 15:36

ids = re.compile(r"/(\d+)") # http://www.view.sdu.edu.cn/info/1003/104956.htm 匹配其中的 1003 和 104956

In [5]:
# 获取网页的文本
def getHtmlText(url):
    try:
        print("getting text:" + url,end=' ')     ###
        
        r = requests.get(url)
        
        r.raise_for_status()                    # 检查响应码

        r.encoding = r.apparent_encoding        # 调整编码
        
        visited_URL_set.add(url)                ## 记录访问过的网址
        
        print(" ------------------Completed!")  ###
        
        return r.text
    
    except Exception as e:
        
        print(" ------------------Dismissed!")  ###
        print(e)
        
        return ""                               #保持程序继续往下执行，返回空串

In [6]:

# 从网页的文本中获取需要继续爬取的网址
def getAllAnchors(url,text):
    if not text:
        return set()
    
    soup = BeautifulSoup(text,"html.parser")
    
    tmpList = soup.find_all("a") 
    
    result = set([])
    for a in tmpList:
        if a.has_attr("href"):
            # url : http://www.view.sdu.edu.cn/info/1003/104085.htm  --> http://www.view.sdu.edu.cn/info/1003/
            if url.endswith(".htm"):
                url = url[:url.rfind("/")]
                
            if re.match(r"^[a-zA-Z0-9]*\.htm$",a["href"]):
                result.add(url+"/"+a["href"])
                
            elif a["href"].startswith("http://www.view.sdu.edu.cn/"):
                result.add(a["href"])
                
            elif re.match(r"^info/\d{4}/\w*\.htm$",a["href"]):
                result.add(r"http://www.view.sdu.edu.cn/"+a["href"])

    return result

In [7]:
# -----测试---- 标签的获取
if test_version:
    text = getHtmlText(r"http://www.view.sdu.edu.cn/")
    #print(text[:200])
    anchors = getAllAnchors(r"http://www.view.sdu.edu.cn/",text)

In [8]:
# 获取网页的正文
def getMainBody(url,text):
    if not text:
        return ""
    try:
        soup = BeautifulSoup(text,"html.parser")
        
        # 获取新闻元信息
        div_news_tit = soup.find("div",class_="news_tit")

        title = div_news_tit.h3.string      #标题

        p = div_news_tit.find_all("p")[1] 
        
        time = p.contents[0].string.strip() #时间
        time = time_pattern.findall(time)[0]      

        # http://www.view.sdu.edu.cn/info/1010/47862.htm
        # 新闻的ID
        news_ids = ids.findall(url)  #[1010,47862]
        
        news_id = "_".join(news_ids) #1010_47862
        
        #点击数不是静态的无法获取
        dynamic_click = r"http://www.view.sdu.edu.cn/system/resource/code/news/click/dynclicks.jsp?clickid="+ news_ids[1] + "&owner=1251758245&clicktype=wbnews"
        
        r = requests.get(dynamic_click)
        cick_times = int(r.text)
        
        # 获取新闻内容
        div_news_content = soup.find("div",class_="news_content")
           
        content = "".join(div_news_content.stripped_strings)
        #re.sub(r"\s",'',content) #去掉空白字符
        
        return news_id,title,time,cick_times,content
    except Exception as e:
        print(e)
        return "","","",0,""

In [9]:
# 保存每个新闻
def saveNews(save_path,url,news_id,title,time,cick_times,content):   
    if len(content) < 10:
        return
    
    if not os.path.exists(save_path):
        os.makedirs(save_path)

    filename = os.path.join(save_path,news_id+".txt")

    with codecs.open(filename,"w","utf-8") as f:
        f.write(title+"\n")
        f.write(time+"\n")
        f.write(str(cick_times)+"\n")
        f.write(url+"\n")
        f.write(content)

In [10]:
# ---- 测试 ---- 新闻内容的获取
if test_version:
    testUrl1 = r"http://www.view.sdu.edu.cn/info/1003/104085.htm" #用P标签包裹了新闻内容
    testUrl2 = r"http://www.view.sdu.edu.cn/info/1010/47862.htm" # 用dic包裹了新闻内容
    testUrl3 = r"http://www.view.sdu.edu.cn/info/1018/39084.htm" #用p
    
    runUrl = testUrl3
    text = getHtmlText(runUrl)
    res = getMainBody(runUrl,text)
    print(res)
    saveNews(r"G:/trash/temp",runUrl,*res)

In [11]:
def runner(url = r"http://www.view.sdu.edu.cn"):
    try:
        if len(visited_URL_set) >= MAX_NUM_NEWS:
            return
        
        #获取网页文本
        text = getHtmlText(url)
        #获取其中的URL
        anchors = getAllAnchors(url,text)
        
        if re.match(r"http://www.view.sdu.edu.cn/info/",url):
            
            #从文本中获取新闻信息
            newsBean = getMainBody(url,text)
            saveNews(savepath,url,*newsBean)
            
        for sub_url in anchors:
            if sub_url not in visited_URL_set:
                runner(sub_url)
                
    except Exception as e:
        print(e)
        return

In [12]:
if not test_version:
    runner()

getting text:http://www.view.sdu.edu.cn  ------------------Completed!
getting text:http://www.view.sdu.edu.cn/sdjx.htm  ------------------Completed!
getting text:http://www.view.sdu.edu.cn/info/1023/105218.htm  ------------------Completed!
getting text:http://www.view.sdu.edu.cn/info/1021/104998.htm  ------------------Completed!
getting text:http://www.view.sdu.edu.cn/info/1021/104995.htm  ------------------Completed!
getting text:http://www.view.sdu.edu.cn/info/1207/104940.htm  ------------------Completed!
getting text:http://www.view.sdu.edu.cn/info/1021/105267.htm  ------------------Completed!
getting text:http://www.view.sdu.edu.cn/info/1021/105258.htm  ------------------Completed!
getting text:http://www.view.sdu.edu.cn/info/1003/104956.htm  ------------------Completed!
getting text:http://www.view.sdu.edu.cn/info/1003/104879.htm  ------------------Completed!
getting text:http://www.view.sdu.edu.cn/info/1022/105086.htm  ------------------Completed!
getting text:http://www.view.sdu

getting text:http://www.view.sdu.edu.cn/info/1021/101864.htm  ------------------Completed!
getting text:http://www.view.sdu.edu.cn/info/1020/101949.htm  ------------------Completed!
getting text:http://www.view.sdu.edu.cn/info/1020/101965.htm  ------------------Completed!
getting text:http://www.view.sdu.edu.cn/info/1020/101964.htm  ------------------Completed!
getting text:http://www.view.sdu.edu.cn/info/1003/94617.htm  ------------------Completed!
getting text:http://www.view.sdu.edu.cn/info/1003/94570.htm  ------------------Completed!
getting text:http://www.view.sdu.edu.cn/info/1003/94486.htm  ------------------Completed!
getting text:http://www.view.sdu.edu.cn/info/1011/96041.htm  ------------------Completed!
'NoneType' object has no attribute 'h3'
getting text:http://www.view.sdu.edu.cn/info/1003/94171.htm  ------------------Completed!
getting text:http://www.view.sdu.edu.cn/info/1016/96320.htm  ------------------Completed!
getting text:http://www.view.sdu.edu.cn/info/1003/90462.

getting text:http://www.view.sdu.edu.cn/info/1003/92411.htm  ------------------Completed!
getting text:http://www.view.sdu.edu.cn/info/1003/94105.htm  ------------------Completed!
getting text:http://www.view.sdu.edu.cn/info/1003/94440.htm  ------------------Completed!
getting text:http://www.view.sdu.edu.cn/info/1022/92456.htm  ------------------Completed!
getting text:http://www.view.sdu.edu.cn/info/1003/94864.htm  ------------------Completed!
getting text:http://www.view.sdu.edu.cn/info/1021/93824.htm  ------------------Completed!
getting text:http://www.view.sdu.edu.cn/info/1023/92358.htm  ------------------Completed!
getting text:http://www.view.sdu.edu.cn/info/1003/92225.htm  ------------------Completed!
getting text:http://www.view.sdu.edu.cn/info/1003/92775.htm  ------------------Completed!
getting text:http://www.view.sdu.edu.cn/info/1003/91851.htm  ------------------Completed!
getting text:http://www.view.sdu.edu.cn/info/1003/91819.htm  ------------------Completed!
getting te

getting text:http://www.view.sdu.edu.cn/info/1004/76022.htm  ------------------Completed!
getting text:http://www.view.sdu.edu.cn/info/1023/90842.htm  ------------------Completed!
getting text:http://www.view.sdu.edu.cn/info/1003/91643.htm  ------------------Completed!
getting text:http://www.view.sdu.edu.cn/info/1011/74855.htm  ------------------Completed!
'NoneType' object has no attribute 'h3'
getting text:http://www.view.sdu.edu.cn/info/1022/91625.htm  ------------------Completed!
getting text:http://www.view.sdu.edu.cn/info/1021/103714.htm  ------------------Completed!
getting text:http://www.view.sdu.edu.cn/info/1021/102434.htm  ------------------Completed!
getting text:http://www.view.sdu.edu.cn/info/1022/104752.htm  ------------------Completed!
getting text:http://www.view.sdu.edu.cn/info/1021/102548.htm  ------------------Completed!
getting text:http://www.view.sdu.edu.cn/info/1023/75034.htm  ------------------Completed!
getting text:http://www.view.sdu.edu.cn/info/1023/76824.

list index out of range
getting text:http://www.view.sdu.edu.cn/info/1015/91684.htm  ------------------Completed!
list index out of range
getting text:http://www.view.sdu.edu.cn/info/1003/70803.htm  ------------------Completed!
getting text:http://www.view.sdu.edu.cn/info/1023/90698.htm  ------------------Completed!
getting text:http://www.view.sdu.edu.cn/info/1003/92627.htm  ------------------Completed!
getting text:http://www.view.sdu.edu.cn/info/1022/76367.htm  ------------------Completed!
getting text:http://www.view.sdu.edu.cn/info/1023/65686.htm  ------------------Completed!
getting text:http://www.view.sdu.edu.cn/info/1023/76789.htm  ------------------Completed!
getting text:http://www.view.sdu.edu.cn/info/1023/91304.htm  ------------------Completed!
getting text:http://www.view.sdu.edu.cn/info/1023/73242.htm  ------------------Completed!
getting text:http://www.view.sdu.edu.cn/info/1022/66627.htm  ------------------Completed!
getting text:http://www.view.sdu.edu.cn/info/1003/72

...

getting text:http://www.view.sdu.edu.cn/info/1018/39083.htm  ------------------Completed!
getting text:http://www.view.sdu.edu.cn/info/1018/39138.htm  ------------------Completed!
getting text:http://www.view.sdu.edu.cn/info/1018/96483.htm  ------------------Completed!
getting text:http://www.view.sdu.edu.cn/info/1018/95705.htm  ------------------Completed!
getting text:http://www.view.sdu.edu.cn/info/1018/39126.htm  ------------------Completed!
getting text:http://www.view.sdu.edu.cn/info/1018/47975.htm  ------------------Completed!
getting text:http://www.view.sdu.edu.cn/info/1018/39077.htm  ------------------Completed!
getting text:http://www.view.sdu.edu.cn/info/1018/91000.htm  ------------------Completed!
getting text:http://www.view.sdu.edu.cn/info/1018/39084.htm  ------------------Completed!
getting text:http://www.view.sdu.edu.cn/info/1018/72980.htm  ------------------Completed!
getting text:http://www.view.sdu.edu.cn/info/1018/47980.htm  ------------------Completed!
getting te

getting text:http://www.view.sdu.edu.cn/info/1022/101757.htm  ------------------Completed!
getting text:http://www.view.sdu.edu.cn/info/1022/101525.htm  ------------------Completed!
getting text:http://www.view.sdu.edu.cn/info/1022/101701.htm  ------------------Completed!
getting text:http://www.view.sdu.edu.cn/info/1022/102377.htm  ------------------Completed!
getting text:http://www.view.sdu.edu.cn/info/1022/102599.htm  ------------------Completed!
getting text:http://www.view.sdu.edu.cn/info/1022/104083.htm  ------------------Completed!
getting text:http://www.view.sdu.edu.cn/info/1022/104076.htm  ------------------Completed!
getting text:http://www.view.sdu.edu.cn/info/1022/101315.htm  ------------------Completed!
getting text:http://www.view.sdu.edu.cn/info/1023/104608.htm  ------------------Completed!
getting text:http://www.view.sdu.edu.cn/info/1022/104891.htm  ------------------Completed!
getting text:http://www.view.sdu.edu.cn/info/1022/102360.htm  ------------------Completed!

getting text:http://www.view.sdu.edu.cn/info/1011/37861.htm  ------------------Completed!
getting text:http://www.view.sdu.edu.cn/info/1011/37818.htm  ------------------Completed!
getting text:http://www.view.sdu.edu.cn/info/1011/37848.htm  ------------------Completed!
getting text:http://www.view.sdu.edu.cn/info/1011/37867.htm  ------------------Completed!
getting text:http://www.view.sdu.edu.cn/info/1011/37789.htm  ------------------Completed!
'NoneType' object has no attribute 'h3'
getting text:http://www.view.sdu.edu.cn/info/1011/37845.htm  ------------------Completed!
getting text:http://www.view.sdu.edu.cn/info/1011/37791.htm  ------------------Completed!
getting text:http://www.view.sdu.edu.cn/info/1011/37827.htm  ------------------Completed!
getting text:http://www.view.sdu.edu.cn/info/1011/37764.htm  ------------------Completed!
'NoneType' object has no attribute 'h3'
getting text:http://www.view.sdu.edu.cn/info/1011/37826.htm  ------------------Completed!
getting text:http://

getting text:http://www.view.sdu.edu.cn/info/1020/104982.htm  ------------------Completed!
getting text:http://www.view.sdu.edu.cn/info/1026/47999.htm  ------------------Completed!
list index out of range
getting text:http://www.view.sdu.edu.cn/new/uploadfile/2016/0622/cunzai.mp4  ------------------Dismissed!
404 Client Error: Not Found for url: http://www.view.sdu.edu.cn/new/uploadfile/2016/0622/cunzai.mp4
getting text:http://www.view.sdu.edu.cn/info/1026/74544.htm  ------------------Completed!
list index out of range
getting text:http://www.view.sdu.edu.cn/info/1026/74546.htm  ------------------Completed!
list index out of range
getting text:http://www.view.sdu.edu.cn/info/1026/44150.htm  ------------------Completed!
list index out of range
getting text:http://www.view.sdu.edu.cn/info/1026/74547.htm  ------------------Completed!
list index out of range
getting text:http://www.view.sdu.edu.cn/info/1026/43649.htm  ------------------Completed!
list index out of range
getting text:http:/

getting text:http://www.view.sdu.edu.cn/info/1022/104902.htm  ------------------Completed!
getting text:http://www.view.sdu.edu.cn/info/1023/104781.htm  ------------------Completed!
getting text:http://www.view.sdu.edu.cn//info/1022/104437.htm  ------------------Completed!
getting text:http://www.view.sdu.edu.cn/info/1022/104926.htm  ------------------Completed!
getting text:http://www.view.sdu.edu.cn//info/1022/104513.htm  ------------------Completed!
getting text:http://www.view.sdu.edu.cn/info/1022/104782.htm  ------------------Completed!
getting text:http://www.view.sdu.edu.cn/info/1022/104534.htm  ------------------Completed!
getting text:http://www.view.sdu.edu.cn//info/1022/104438.htm  ------------------Completed!
getting text:http://www.view.sdu.edu.cn/info/1023/104990.htm  ------------------Completed!
getting text:http://www.view.sdu.edu.cn/info/1023/105229.htm  ------------------Completed!
'NoneType' object has no attribute 'h3'
getting text:http://www.view.sdu.edu.cn/info/10

In [13]:
len(visited_URL_set)

1848